In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# define hyperparameters
batch_size = 32
learning_rate = 1e-2
num_epochs = 50

# download MNIST dataset
train_dataset = datasets.FashionMNIST(
    root='./datasets', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.FashionMNIST(
    root='./datasets', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(f'size of training set is: {len(train_loader.dataset)}')
print(f'size of testing set is: {len(test_loader.dataset)}')

# define model: a 3-layer fully connected neural network
class FullyConnectedNN(nn.Module):
    def __init__(self, in_dim, n_layer1, n_layer2, out_dim):
        super(FullyConnectedNN, self).__init__()
        self.layer1 = nn.Linear(in_dim, n_layer1)
        self.layer2 = nn.Linear(n_layer1, n_layer2)
        self.layer3 = nn.Linear(n_layer2, out_dim)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x
        
model = FullyConnectedNN(28 * 28, 300, 100, 10)  # image size is 28x28
use_gpu = torch.cuda.is_available()
if use_gpu:
    model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


size of training set is: 60000
size of testing set is: 10000


In [4]:
for epoch in range(num_epochs):
    # train model
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        img = img.view(img.size(0), -1)
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss
        _, predict = torch.max(out, 1)
        running_acc += (predict == label).float().mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i%300 == 0:
            print(f'[{epoch+1}/{num_epochs}] Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    print(f'Finish {epoch+1} epochs, Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
       
    # test model
    model.eval()
    eval_loss = 0
    eval_acc = 0
    for data in test_loader:
        img, label = data
        img = img.view(img.size(0), -1)
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        with torch.no_grad():
            out = model(img)
            loss = criterion(out, label)
        eval_loss += loss
        _, predict = torch.max(out, 1)
        eval_acc += (predict == label).float().mean()

    print(f'Test Loss: {eval_loss/len(test_loader):.6f}, Acc: {eval_acc/len(test_loader):.6f}')

[1/50] Loss: 1.686254, Acc: 0.540104
[1/50] Loss: 1.329801, Acc: 0.605365
[1/50] Loss: 1.148804, Acc: 0.644132
[1/50] Loss: 1.038376, Acc: 0.670807
[1/50] Loss: 0.957945, Acc: 0.693000
[1/50] Loss: 0.900012, Acc: 0.709323
Finish 1 epochs, Loss: 0.887772, Acc: 0.713000
Test Loss: 0.603589, Acc: 0.785244
[2/50] Loss: 0.569670, Acc: 0.804688
[2/50] Loss: 0.557351, Acc: 0.808542
[2/50] Loss: 0.541870, Acc: 0.814826
[2/50] Loss: 0.539221, Acc: 0.814453
[2/50] Loss: 0.538766, Acc: 0.814417
[2/50] Loss: 0.534279, Acc: 0.815486
Finish 2 epochs, Loss: 0.531916, Acc: 0.816300
Test Loss: 0.549383, Acc: 0.795927
[3/50] Loss: 0.491340, Acc: 0.832813
[3/50] Loss: 0.481378, Acc: 0.835885
[3/50] Loss: 0.484162, Acc: 0.833542
[3/50] Loss: 0.485529, Acc: 0.832656
[3/50] Loss: 0.482012, Acc: 0.832792
[3/50] Loss: 0.483350, Acc: 0.831597
Finish 3 epochs, Loss: 0.482812, Acc: 0.831767
Test Loss: 0.498946, Acc: 0.823482
[4/50] Loss: 0.447139, Acc: 0.844896
[4/50] Loss: 0.450350, Acc: 0.840104
[4/50] Loss: 0

[27/50] Loss: 0.384263, Acc: 0.865868
Finish 27 epochs, Loss: 0.385146, Acc: 0.865517
Test Loss: 0.454036, Acc: 0.840755
[28/50] Loss: 0.385761, Acc: 0.863333
[28/50] Loss: 0.385005, Acc: 0.863490
[28/50] Loss: 0.382630, Acc: 0.864306
[28/50] Loss: 0.381408, Acc: 0.864844
[28/50] Loss: 0.384652, Acc: 0.864729
[28/50] Loss: 0.384399, Acc: 0.864792
Finish 28 epochs, Loss: 0.384515, Acc: 0.864733
Test Loss: 0.443637, Acc: 0.843051
[29/50] Loss: 0.373580, Acc: 0.864688
[29/50] Loss: 0.375713, Acc: 0.865417
[29/50] Loss: 0.377111, Acc: 0.864792
[29/50] Loss: 0.375763, Acc: 0.865234
[29/50] Loss: 0.378349, Acc: 0.864917
[29/50] Loss: 0.382894, Acc: 0.863906
Finish 29 epochs, Loss: 0.383702, Acc: 0.863600
Test Loss: 0.439544, Acc: 0.843650
[30/50] Loss: 0.379596, Acc: 0.865417
[30/50] Loss: 0.387234, Acc: 0.864010
[30/50] Loss: 0.383817, Acc: 0.865938
[30/50] Loss: 0.382686, Acc: 0.865755
[30/50] Loss: 0.383536, Acc: 0.865292
[30/50] Loss: 0.383385, Acc: 0.865625
Finish 30 epochs, Loss: 0.383